# Turbofan Tutorial
`pyturbo`  library is provided by twiinIT to assembly a simple turbofan system.

The library is made of components: 

- `compressor` : fluid out is computed from fluid in and power provided by shaft with constant efficiency. 
- `combustor` : combustion is made considering constant FHV.
- `turbine` : power is extracted from fluid in considering a given expansion ratio and constant efficiency. 
- `inlet` and `nozzle` are computing `drag` and `thrust` from fluid conditions, ambiant pressure and throat section. 
- `nacelle`: envelop over the engine.
- `ogv`, `intermediate_casing` and `trf` and structures with aero channels. 

They are numerical components:

- `fluid_spitter` is used to split the flow into primary and secondary flow
- `shaft_spitter` is used to split the shaft power into booster and fan compressor

Aero 0D and simplified geometry are considered. 

A turbofan system is generated.

In [ ]:
from pyturbo.systems.turbofan import TurbofanWithAtm
sys = TurbofanWithAtm("sys")
sys.run_once()
tf = sys.tf

In [ ]:
# geometrical view
tf.occ_view.get_value().render()

# Simulation

The turbofan system has a couple of equations/unknowns to solve. We use `cosapp` non-linear solver for this purpose.

In [ ]:
from cosapp.drivers import NonLinearSolver

## Direct mode
`thrust` is computed from `fuel_W`.

In [ ]:
# off-design solver
solver_direct_mode = sys.add_driver(NonLinearSolver('solver_direct_mode', tol = 1e-6))

In [ ]:
%%time
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

# use case
tf.fuel_W = .5

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', tf.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

## Control mode
`fuel_W` is computed to match functional request, here the fan rotational speed `N1` value.

In [ ]:
# control solver
sys.drivers.clear()
solver_control_mode = sys.add_driver(NonLinearSolver('solver_control_mode', tol=1e-6))
solver_control_mode.add_unknown('tf.fuel_W', max_rel_step = 0.5)
solver_control_mode.add_target('tf.N1')

In [ ]:
%%time
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

# use case
tf.N1 = 5000.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

# Design

Turbofan design characteristics are related to components and physical properties.

In [ ]:
from pathlib import Path
import pyturbo.systems.turbofan.data as tf_data
from pyturbo.utils import load_from_json

## Update geometry

In [ ]:
from pathlib import Path
import pyturbo.systems.turbofan.data as tf_data
from pyturbo.utils import load_from_json

data_geom = Path(tf_data.__file__).parent / "CFM56_7_geom.json"
load_from_json(tf, data_geom)

# design solver
sys.drivers.clear()
sys.add_driver(solver_direct_mode)

In [ ]:
%%time
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

In [ ]:
tf.occ_view.get_value().render()

## Change state of the art

In [ ]:
data_design_data = Path(tf_data.__file__).parent / "CFM56_7_design_data.json"
load_from_json(tf, data_design_data)

# design solver
sys.drivers.clear()
solver_scaling = sys.add_driver(NonLinearSolver('solver_scaling', tol=1e-6))

# design method
solver_scaling.extend(tf.design_methods['scaling'])

# engine
tf.fan_module.fan.aero.utip = 420.
tf.fan_module.booster.aero.psi = 0.35

tf.turbine.aero.psi = 1.25

tf.core.compressor.aero.psi = 0.21
tf.core.compressor.aero.utip = 420.
tf.core.turbine.aero.psi = 1.2
tf.core.combustor.aero.Tcomb = 1700.

tf.pr_nozzle = 1.1

In [ ]:
%%time
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

In [ ]:
tf.occ_view.get_value().render()

## Change dimension

In [ ]:
# solver
solver_scaling.extend(tf.design_methods['tuning_thrust'])
solver_scaling.extend(tf.design_methods['tuning_bpr'])

In [ ]:
%%time
# engine
sys.thrust = 90e3
tf.bpr = 6.0

# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

In [ ]:
tf.occ_view.get_value().render()

## Off-design computation after design

Fuel consumption for a given altitude/mach/dtamb and thrust. 

In [ ]:
# off-design mode
sys.drivers.clear()
run = sys.add_driver(solver_control_mode)

In [ ]:
%%time
# use case
tf.N1 *= 0.9

# environment conditions
sys.altitude = 10000.
sys.mach = 0.8
sys.dtamb = 0.

# run 
sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fuel flow =', round(tf.fuel_W, 1), 'kg/s')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))